In [ ]:
# doc for splitting png/ tif files into small tiles for CNN input

In [ ]:
# for RGB


# ITS BAD TO RELY ON INDEXES TO MATCH UP  so need to sort the files when i open the directroy
# list index out of range means i have called the file the wrong name, like got CN box1 and CN box 8 mixed up



#split data
# WORKS !!!!

import gdal
import image_slicer
import math
import os
from PIL import Image
from skimage.transform import resize   
import numpy as np  


# Split the Images
def split_image(dim_pix, im):
    # Find the number of sub-Images that fit in rows
    rows = []
    tiles = []
    for i in range((math.floor(im.shape[0] / dim_pix))):
        rows.append(i)
    # Find the number of sub-Images that fit in rows
    columns = []
    for i in range((math.floor(im.shape[1] / dim_pix))):
        columns.append(i)

    # Numerically identify the sub-Images
    a = 0
    for i in rows:
        for j in columns:
            # Check for 244 x 244 (Mask) or 244 x 244 x 3 (TC Images)
            if (im[0 + (dim_pix * j): dim_pix + (dim_pix * j),
                  0 + dim_pix * i: dim_pix + (dim_pix * i)].shape[0]) == dim_pix:
                if (im[0 + (dim_pix * j): dim_pix + (dim_pix * j),
                  0 + dim_pix * i: dim_pix + (dim_pix * i)].shape[1]) == dim_pix:

                    tile = im[0 + (dim_pix * j): dim_pix + (dim_pix * j),
                            0 + dim_pix * i: dim_pix + (dim_pix * i)]

                    # Stop white tiles for positive results
                    count = np.count_nonzero(tile == 1) == (dim_pix * dim_pix)
                    if count:
                        all_black = np.tile(1, (dim_pix, dim_pix))
                        tiles.append(tile)
                    else:
                        tiles.append(tile)
                    a += 1
                else:
                    print("Out of shape")
    return tiles


######################################################################

# set dir
label_dir = sorted(os.listdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/Labels')) # FINAL/ALL_snow_labelled
img_dir = sorted(os.listdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/RGB')) # *.tif Data_collection/FINAL/ALL_stacked_tifs


import glob
import os 
from PIL import Image
import sys

import earthpy.plot as ep
import earthpy.spatial as es
import rasterio as rio
import cv2




# rescale between 0-255
# pixel height / (max pixel height) gives 0-1
# then * 255 
# then make an integer and save?


input_tensor_dimensions = 244


for i in range(len(label_dir)):
    i_label = label_dir[i]
   # print("labelled file name", i_label)
    # split the individual files in dir
    name = i_label.split(".")[0]
    print("label name", name)
    os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/Labels') # FINAL/ALL_snow_labelled
    # read png labelled pics in as array  
    label_png = np.array(Image.open(i_label))
    array = label_png
   # print("orig unique", (np.unique(array)))
    #print("shape of orig arr", array.shape[0])
    # make all snow 1 values into 255. pics should be only 0 for black and 255 for white 
    # normalise_array = array * 255
    
    # delete 30 rows and 30 columns from first and last to remove white space before splitting
    # array, row/col, axix i.e. 0/1 for row/col
    
    np_first_row_del = np.delete(array, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29], 0)
    np_first_col_del = np.delete(np_first_row_del, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29], 1)
    
    np_last_row = np.delete(np_first_col_del, [-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12,-13,-14,-15,-16,-17,-18,-19,-20,-21,-22,-23,-24,-25,-26,-27,-28,-29], 0)
    np_no_whitespace = np.delete(np_last_row, [-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12,-13,-14,-15,-16,-17,-18,-19,-20,-21,-22,-23,-24,-25,-26,-27,-28,-29], 1)
   # print("npppppp", np_no_whitespace)
    
    
   # normalise_array = array * 255
    array_height = int((np_no_whitespace.shape[0] / 244)) * 244 # int((array.shape[0] / 244)) * 244
    #print("shape of resized arr", array.shape[0])
    array_width = int((np_no_whitespace.shape[1] / 244)) * 244 # / array.shape[1])) * 244?

    
    # all pixels are resized to 0-1 after resize function??
    resized_array = resize(np_no_whitespace, (array_height, array_width))
    

   # print("unique rezized:", np.unique(resized_array))
    # Check for 244 x 244 (Mask) or 244 x 244 x 3 (TC Images)
    label_tiles = split_image(dim_pix=input_tensor_dimensions, im=resized_array) # resized_array
    #print(label_tiles)
    n = len(label_tiles)
   # print("n", n)
   # if n!= min(array_height/244,array_width/244) **2:
    #    print(name)

                
    os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/To_ribble/RGB/NDSI_DEM_NIR/PNGLabels') #save path to split labelled pic
    for index in range(n):
        print("label ind", index)
        # name the output labelled pic and save as png file 
        outfile_png = name + "_" + str(index+1) + ".png"
       # print("ind png:", index)
       # print("out file png", outfile_png)
        # make array of output file
        # HERE MY PICS SHOULD BE 1 FOR WHITE AND 0 FOR BLACK
        im = Image.fromarray((label_tiles[index] * 255).astype(np.uint8)) #* 255

       # im = Image.fromarray((label_tiles[index]).astype(np.uint8)) # this makes values 255 * 255
       # print("UNIQUE NO im.", np.unique(im))
       # print("Minimum value in the whole RESIZED array:%d"%(im.min()))
       # print("Maximum value in the whole RESIZED array:%d"%(im.max()))
      #  print("im x 255", im)
        # save output 
        im.save(outfile_png) #HERE TO SAVE SPLIT LABELLED PIC
    
    
    
    
    # the 3 channnel pic. (NIR, DEM, NDSI) pic
    rgb_image = "{}{}".format(name, ".png") # change to .tif !!!!!!
    
    i_image = "/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/RGB/{}".format(rgb_image) # FINAL/ALL_stacked_tifs/
    print("rgb name", i_image)
    #print("rgb image file name",i_image)
    os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/RGB') # Data_collection/FINAL/ALL_stacked_tifs
   
    # print("i image", i_image)
   # img_rgb = Image.Open(i_image).convert("RGB") # gdal.Open(i_image)
    #  OpenCV, OpenCV for Python treats image data as ndarray
    rgb_array = cv2.imread(i_image)
    
     #redChannel = rgbImage(:,:,0); % Red channel
   # greenChannel = rgbImage(:,:,1); % Green channel
   # blueChannel = rgbImage(:,:,2); % Blue channel
    
    # remove whitespace (first and last 30 rows/columns)
    np_first_row = np.delete(rgb_array, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29], 0)
    np_first_col = np.delete(np_first_row, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29], 1)
    
    
    np_last_ro = np.delete(np_first_col, [-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12,-13,-14,-15,-16,-17,-18,-19,-20,-21,-22,-23,-24,-25,-26,-27,-28,-29], 0)
    np_rm_whitespace = np.delete(np_last_ro, [-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12,-13,-14,-15,-16,-17,-18,-19,-20,-21,-22,-23,-24,-25,-26,-27,-28,-29], 1)
   # print("npppppp", np_rm_whitespace)
    
    # seperate bands first 
   # redChannel = rgbImage(:,:,0); % Red channel
   # greenChannel = rgbImage(:,:,1); % Green channel
   # blueChannel = rgbImage(:,:,2); % Blue channel
    
   # print("0", rgb_array.shape[0])
   # print("1", rgb_array.shape[1])
   # print("2", rgb_array.shape[2])
   # img_rgb = img_rgb.convert("RGB")
   # print("image rgb", img_rgb)
   # rgb_array = img_rgb.ReadAsArray()
    #print("unique in orig RGB", (np.unique(array)))
    # Resize the array for the correct splitting dimensions for a tif 
    rgb_array_height = int((np_rm_whitespace.shape[0] / 244)) * 244 # all pixels are resized to 0-1 after resize function
    print("tif resized array_height", rgb_array_height)
    rgb_array_width = int((np_rm_whitespace.shape[1] / 244)) * 244
    print("tif resize array_width", rgb_array_width)
    
    # REDEFINE THIS TO TIF DIMENSIONS
    resized_array = resize(np_rm_whitespace, (rgb_array_height, rgb_array_width, 3)) # all pixels resized to
   # print("shape", np.shape(resized_array))
    
    print("unique in resized RGB", (np.unique(resized_array)))
    #print("tif arr", resized_array)
    #print("Minimum value in the whole RESIZED tif array:%d"%(resized_array.min()))
    #print("Maximum value in the whole RESIZEDtif  array:%d"%(resized_array.max()))
    
    # CANT JUST SAVE AS .TF- HAVE TO WRITE TIF FILE 
    
    basemap_tiles = split_image(dim_pix=input_tensor_dimensions, im=resized_array)
   # print("len basemap tiles", len(basemap_tiles))
    os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/To_ribble/RGB/NDSI_DEM_NIR/JPEGImages') #save path Data_collection/FINAL/test_tif
    for ind in range(len(label_tiles)): #
        outfile_rgb = name + "_" + str(ind+1) + ".png"
        print("rgb ind", ind)
 
        # Note that if the pixel value is represented by 0.0 to 1.0, it is necessary to multiply by 255 and convert to uint8 and save.
        # https://note.nkmk.me/en/python-numpy-image-processing/ 
        im = Image.fromarray((basemap_tiles[ind] * 255).astype(np.uint8))
       # print("immmmm", im)
       # print("range im", (np.unique(im)))
      
        im.save(outfile_rgb)
        



In [ ]:
# this splits geotiffs and saves the mini tiles with meta data

import os
from itertools import product
import rasterio as rio
from rasterio import windows



in_path = '/Users/emilybirch/Documents/UCL_Dissertation/Final_files/trial_split/tif'
input_filename = 'Al_snow_1.tif'
#in_path2 = os.listdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/trial_split/tif')


out_path = '/Users/emilybirch/Documents/UCL_Dissertation/Final_files/trial_split/split_tif'
# output_filename = input_filename +'{}.tif' #'Al_snow_1_{}-{}



#for file in in_path2:
#    input_filename = file
#    print("in", input_filename)
    #name = input_filename.split(".")[0]
    
def get_tiles(ds, width=244, height=244):
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in  offsets:
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform


with rio.open(os.path.join(in_path, input_filename)) as inds:
    tile_width, tile_height = 244, 244

    meta = inds.meta.copy()
    i = 0
    for window, transform in get_tiles(inds):
        i += 1
        output_filename = '{}_'.format(i) + input_filename 
        print("out name", output_filename)
        print(window)
        meta['transform'] = transform
        meta['width'], meta['height'] = window.width, window.height
        outpath = os.path.join(out_path,output_filename.format(int(window.col_off), int(window.row_off)))
        with rio.open(outpath, 'w', **meta) as outds:
            outds.write(inds.read(window=window))

            


In [48]:
# splitting for NDSI_NIR_DEM
import os
import numpy as np
from PIL import Image
import glob
import os 
from PIL import Image
import sys
import gdal
import image_slicer
import math
from skimage.transform import resize   
import imageio 
import earthpy.plot as ep
import earthpy.spatial as es
import rasterio as rio
import cv2
 


# Split the Images
def split_image(dim_pix, im):
    # Find the number of sub-Images that fit in rows
    rows = []
    tiles = []
    for i in range((math.floor(im.shape[0] / dim_pix))):
        rows.append(i)
    # Find the number of sub-Images that fit in rows
    columns = []
    for i in range((math.floor(im.shape[1] / dim_pix))):
        columns.append(i)

    # Numerically identify the sub-Images
    a = 0
    for i in rows:
        for j in columns:
            # Check for 244 x 244 (Mask) or 244 x 244 x 3 (TC Images)
            if (im[0 + (dim_pix * j): dim_pix + (dim_pix * j),
                  0 + dim_pix * i: dim_pix + (dim_pix * i)].shape[0]) == dim_pix:
                if (im[0 + (dim_pix * j): dim_pix + (dim_pix * j),
                  0 + dim_pix * i: dim_pix + (dim_pix * i)].shape[1]) == dim_pix:

                    tile = im[0 + (dim_pix * j): dim_pix + (dim_pix * j),
                            0 + dim_pix * i: dim_pix + (dim_pix * i)]

                    # Stop white tiles for positive results
                    count = np.count_nonzero(tile == 1) == (dim_pix * dim_pix)
                    if count:
                        all_black = np.tile(1, (dim_pix, dim_pix))
                        tiles.append(tile)
                    else:
                        tiles.append(tile)
                    a += 1
                else:
                    print("Out of shape")
    return tiles


input_tensor_dimensions = 244


label_dir = sorted(os.listdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/Labels')) # FINAL/ALL_snow_labelled
#print(label_dir)
img_dir = sorted(os.listdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/Tifs')) # *.tif Data_collection/FINAL/ALL_stacked_tifs
#print(img_dir)

# label pics:
for i in range(len(label_dir)):
    os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/Labels') # FINAL/ALL_snow_labelled
    # read png labelled pics in as array 
    i_label = label_dir[i]
    name = i_label.split(".")[0]
    #print(i_label)
    array = np.array(Image.open(i_label))
   # print(array)
   # print("arr shape", (np.shape(array)))
    # delete 30 rows and 30 columns from first and last to remove white space before splitting
    np_first_row_del = np.delete(array, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29], 0)
    np_first_col_del = np.delete(np_first_row_del, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29], 1)
    
    np_last_row = np.delete(np_first_col_del, [-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12,-13,-14,-15,-16,-17,-18,-19,-20,-21,-22,-23,-24,-25,-26,-27,-28,-29], 0)
    np_no_whitespace = np.delete(np_last_row, [-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12,-13,-14,-15,-16,-17,-18,-19,-20,-21,-22,-23,-24,-25,-26,-27,-28,-29], 1)
    
    # resize so white space is adjusted to the new 244x244 dimensions
    array_height = int((np_no_whitespace.shape[0] / 244)) * 244 
    array_width = int((np_no_whitespace.shape[1] / 244)) * 244 
    resized_array = resize(array, (array_height, array_width))
    label_tiles = split_image(dim_pix=input_tensor_dimensions, im=resized_array) # resized_array
    n = len(label_tiles)
    # set out path
    os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/To_ribble/NDSI_DEM_NIR/PNGLabels') #save path to split labelled pic
    for index in range(n):
        # check indx matches indx of tifs
        #print("label ind", index)
        outfile_png = name + "_" + str(index+1) + ".png"
        im = Image.fromarray((label_tiles[index] * 255).astype(np.uint8))
        im.save(outfile_png)
    
    # tif pics:
    tif_image = "{}{}".format(name, ".tif") # change to .tif !!!!!!
    
    i_image = "/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/Tifs/{}".format(tif_image) # FINAL/ALL_stacked_tifs/
    print("rgb name", i_image)
    #print("rgb image file name",i_image)
    os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/Tifs') # Data_collection/FINAL/ALL_stacked_tifs
    # OpenCV for Python treats image data as ndarray
   # tif_array = cv2.imread(i_image)
    tif_array = cv2.imread(i_image, -1)
    # compare the original pixelxpixel size w the array once rows/cols have been deleted
   # print("tif orig width", tif_array.shape[0])
   # print("tif orig height", tif_array.shape[1])
    #print("dt", (tif_array.dtype))
    #print("tif arr", tif_array)
    # delete whitespace
    np_first_row = np.delete(tif_array, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29], 0)
    np_first_col = np.delete(np_first_row, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29], 1)
    np_last_ro = np.delete(np_first_col, [-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12,-13,-14,-15,-16,-17,-18,-19,-20,-21,-22,-23,-24,-25,-26,-27,-28,-29], 0)
    np_rm_whitespace = np.delete(np_last_ro, [-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12,-13,-14,-15,-16,-17,-18,-19,-20,-21,-22,-23,-24,-25,-26,-27,-28,-29], 1)

    # resize to fit to new 244x244 with minimal whitespace
    tif_array_height = int((np_rm_whitespace.shape[0] / 244)) * 244 
    #print("tif resized array_height", tif_array_height)
    tif_array_width = int((np_rm_whitespace.shape[1] / 244)) * 244
    #print("tif resize array_width", tif_array_width)
    
    resized_array = resize(np_rm_whitespace, (tif_array_height, tif_array_width, 3)) 
    #print("shape", np.shape(resized_array))
    
    # split the big pics into small 244 x244 tiles
    basemap_tiles = split_image(dim_pix=input_tensor_dimensions, im=resized_array)
   # print("len basemap tiles", len(basemap_tiles))
    os.chdir('/Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/To_ribble/NDSI_DEM_NIR/JPEGImages') #save path Data_collection/FINAL/test_tif
    for ind in range(len(label_tiles)): #
        outfile_tif = name + "_" + str(ind+1) + ".tif"
        print("tif ind", ind)
        
        # save as unit8 type for input into model
        im = Image.fromarray((basemap_tiles[ind] * 255).astype(np.uint8))
      
        #im.save(outfile_rgb)
        imageio.imwrite(outfile_tif, im) #.astype('unit8'))
        #imageio.imwrite(sample_path, (255.0 * image).astype('uint8')) 
    
    
    



rgb name /Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/Tifs/Al_snow_1.tif
tif ind 0
tif ind 1
tif ind 2
tif ind 3
tif ind 4
tif ind 5
tif ind 6
tif ind 7
tif ind 8
tif ind 9
tif ind 10
tif ind 11
tif ind 12
tif ind 13
tif ind 14
tif ind 15
tif ind 16
tif ind 17
tif ind 18
tif ind 19
tif ind 20
tif ind 21
tif ind 22
tif ind 23
tif ind 24
tif ind 25
tif ind 26
tif ind 27
tif ind 28
tif ind 29
tif ind 30
tif ind 31
tif ind 32
tif ind 33
tif ind 34
tif ind 35
tif ind 36
tif ind 37
tif ind 38
tif ind 39
tif ind 40
tif ind 41
tif ind 42
tif ind 43
tif ind 44
tif ind 45
tif ind 46
tif ind 47
tif ind 48
tif ind 49
tif ind 50
tif ind 51
tif ind 52
tif ind 53
tif ind 54
tif ind 55
tif ind 56
tif ind 57
tif ind 58
tif ind 59
tif ind 60
tif ind 61
tif ind 62
tif ind 63
rgb name /Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/Tifs/Al_snow_3.tif
tif ind 0
tif ind 1
tif ind 2
tif ind 3
tif ind 4
tif ind 5
tif ind 6
tif ind 7
tif ind 

rgb name /Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/Tifs/Nor_snow_10.tif
tif ind 0
tif ind 1
tif ind 2
tif ind 3
tif ind 4
tif ind 5
tif ind 6
tif ind 7
tif ind 8
tif ind 9
tif ind 10
tif ind 11
tif ind 12
tif ind 13
tif ind 14
tif ind 15
tif ind 16
tif ind 17
tif ind 18
tif ind 19
tif ind 20
tif ind 21
tif ind 22
tif ind 23
tif ind 24
tif ind 25
tif ind 26
tif ind 27
tif ind 28
tif ind 29
tif ind 30
tif ind 31
tif ind 32
tif ind 33
tif ind 34
tif ind 35
tif ind 36
tif ind 37
tif ind 38
tif ind 39
tif ind 40
tif ind 41
tif ind 42
tif ind 43
tif ind 44
tif ind 45
tif ind 46
tif ind 47
tif ind 48
tif ind 49
tif ind 50
tif ind 51
tif ind 52
tif ind 53
tif ind 54
tif ind 55
tif ind 56
tif ind 57
tif ind 58
tif ind 59
tif ind 60
tif ind 61
tif ind 62
tif ind 63
rgb name /Users/emilybirch/Documents/UCL_Dissertation/Final_files/Train_subset/Large_pics/Tifs/Nor_snow_11.tif
tif ind 0
tif ind 1
tif ind 2
tif ind 3
tif ind 4
tif ind 5
tif ind 6
tif ind 7
tif 